Detección de caras con webcam

In [3]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors

En el bucle de procesamiento, las teclas 'd' y 'e' `permiten respectivamenet cambiar de modelo de detección de caras, y en su caso de máscara de detección del rostro.

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [24]:
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

    # Face detection and eye model setup
imodoF = 1
imodoE = 0

debug = 0

#Set camera resolution
cap.set(3,640);
cap.set(4,480);

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # Show detected facial elements
                if imodoF > 0:
                    for (x, y) in shape:
                        cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)


                # Normalize and show
                # color channels
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
                cv2.imshow("Normalized", NormBGR)


    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    if imodoF == 1 or imodoF == 2:
        cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break


# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0


In [1]:
import cv2
import numpy as np
import FaceNormalizationUtils as faceutils
import FaceDetectors
import random
import time
import pygame

# Global variables
changing_images = False
last_image_change_time = time.time()
image_change_duration = 10  # Change the image every 10 seconds
current_image_index = 0
time_remaining = image_change_duration  # Initialize time_remaining

# Function to handle mouse click events
def on_mouse_click(event, x, y, flags, param):
    global changing_images, last_image_change_time, time_remaining
    if event == cv2.EVENT_LBUTTONDOWN:  # Left mouse button click
        changing_images = True
        last_image_change_time = time.time()
        time_remaining = image_change_duration  # Reset time_remaining

# Normalizador HS
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Inicializa la captura de la webcam
cap = cv2.VideoCapture(0)

# Variables para el manejo de la ruleta de imágenes
image_paths = ['lalaland.jpg', 'blade.png', 'drive.png','ryan-gosling-1655362981.jpg']  
songs_paths =[
    "Y2meta.app - 'City of Stars' (Duet ft. Ryan Gosling, Emma Stone) - La La Land Original Motion Picture Soundtrack (128 kbps).mp3",
    'you-look-lonely-i-can-fix-that-128-ytshorts.savetube.me.mp3',
    'Y2meta.app - Kavinsky - Nightcall letra Ingles_Español (128 kbps).mp3',
    "Y2meta.app - Ryan Gosling - I'm Just Ken (From Barbie The Album) [Official Audio] (128 kbps).mp3",

]

# Initialize roulette_image before entering the while loop
roulette_image = cv2.imread(image_paths[current_image_index])

pygame.mixer.init()

# Create a window to capture mouse events
cv2.namedWindow('Webcam')
cv2.setMouseCallback('Webcam', on_mouse_click)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Calculate time remaining until image change
    time_remaining = max(0, image_change_duration - (time.time() - last_image_change_time))

    # Detección de rostros en el frame de la webcam
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])
  
    if values is not None:
        face = values[0]
        x, y, w, h = face
        if w > 0 and h > 0:
            if roulette_image is not None:
                print("dentro")
                print(roulette_image)
   
                # Calculate the position for the superimposed face image above the detected face
                y_offset = max(0, y - h)  # Place it above the detected face
                x_offset = x

                             # Resize the face image to match the size of the detected face
                resized_face_image = cv2.resize(roulette_image, (w, h))

                cv2.putText(frame, f"Which Ryan Gosling are you?", (x_offset-125, y_offset-25), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 2)
                # Superpose the resized face image on the frame
                frame[y_offset:y_offset + h, x_offset:x_offset + w] = resized_face_image

    
    if changing_images:

        if current_image_index == len(image_paths):
            current_image_index = 0
        if time_remaining == 0:
            print("THE TIME HAS COME")
            print("--------------------------------------")
            print("Indice")
            print(current_image_index)
            
            print("--------------------------------------")
            changing_images = False
            last_image_change_time = time.time()
            current_image_index = random.randint(0, len(image_paths)-1)
            pygame.mixer.music.load(songs_paths[current_image_index])  # Replace with your music file
            pygame.mixer.music.play(-1)
        print("AQUI LLEGAMOS JEFE")
        print("--------------------------------------")
        print("Indice")
        print(current_image_index)
        
        print(image_paths[current_image_index])
        roulette_image = cv2.imread(image_paths[current_image_index])
        
        print(roulette_image)

        
        
        print(roulette_image)
        current_image_index += 1
        print("--------------------------------------")

            

    cv2.imshow('Webcam', frame)

    key = cv2.waitKey(1)
    if key & 0xFF == 27:  # Press Esc to exit
        # Stop the music when your program is done
        pygame.mixer.music.stop()
        break

cap.release()
cv2.destroyAllWindows()


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
dentro
[[[145  74   0]
  [145  74   0]
  [145  75   0]
  ...
  [166  86  15]
  [166  85  18]
  [164  85  18]]

 [[144  75   2]
  [144  75   2]
  [144  76   1]
  ...
  [170  88  11]
  [168  86  14]
  [168  86  14]]

 [[143  74   4]
  [143  74   4]
  [143  74   4]
  ...
  [178  91   3]
  [176  90   6]
  [174  91   6]]

 ...

 [[ 68  33  19]
  [ 68  33  19]
  [ 68  33  19]
  ...
  [220 156 131]
  [214 152 128]
  [211 149 125]]

 [[ 68  33  19]
  [ 68  33  19]
  [ 68  33  19]
  ...
  [215 151 126]
  [210 148 124]
  [206 144 120]]

 [[ 68  33  19]
  [ 68  33  19]
  [ 68  33  19]
  ...
  [213 149 124]
  [207 145 121]
  [203 141 117]]]
dentro
[[[145  74   0]
  [145  74   0]
  [145  75   0]
  ...
  [166  86  15]
  [166  85  18]
  [164  85  18]]

 [[144  75   2]
  [144  75   2]
  [144  76   1]
  ...
  [170  88  11]
  [168  86  14]
  [168  86  14]]

 [[143  74   4]
  [143  74   4]
  [